In [5]:
import os # 경로를 지정하거나 연산에 쓰기 위하여
import re # regular expression 주어진 규칙에 맞는 언어 연산
import pandas as pd # 데이터처리 및 데이터과학을 위한 라이브러리
import tensorflow as tf # 데이터 다운받는 용도로 필요
from tensorflow.keras import utils # 데이터 다운받는 용도로 필요
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html태그를 삭제

# IMBD 데이터 다운로드
data_set = tf.keras.utils.get_file(
    fname = "imdb.tar.gz", # 다운받은 파일의 이름
    origin = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract = True)

def directory_data(directory):
  data = {}
  data["review"] = []
  for file_path in os.listdir(directory):
    with open(os.path.join(directory, file_path), "r", encoding='utf-8') as file:
      data["review"].append(file.read())
  return pd.DataFrame.from_dict(data)
def data(directory):
  pos_df = directory_data(os.path.join(directory, "pos"))
  neg_df = directory_data(os.path.join(directory, "neg"))
  pos_df["sentiment"] = 1
  neg_df["sentiment"] = 0
  return pd.concat([pos_df, neg_df])
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))
imdb_pd = pd.concat([train_df, test_df])
imdb_pd.head()

,review,sentiment
0,This movie is finally out on DVD in Italy (com...,1
1,"I haven't seen the original ""Incredible Journe...",1
2,"First things first, Edison Chen did a fantasti...",1
3,I just read the plot summary and it is the wor...,1
4,The story of Ned Kelly has been enshrouded in ...,1


In [10]:
from nltk.corpus import stopwords # 불용어(stopwords)사전을 가져옴
import nltk
#nltk.download('stopwords') # 불용어를 다운 받음
list_stopwords = set(stopwords.words('english')) # 영어로된 불용어 가져다 집합으로 구성
# 아래의 함수는 리뷰글을 받아서, 쓸데없는 부분을 없애는 것이다.
def preprocessing(review, remove_stopwords = True):
    review_text = BeautifulSoup(review, 'html5lib').get_text() # html 태그제거
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    # 위의 문장은 subtitution, 즉 대체하라는 의미이다.
    # a-z, A-Z를 제외하고(제외한다는 표현은 꺽새 ^를 쓴다) 나머지들은 모두 두 번째 항목인 빈칸으로 대체
    # 어디에서 ? 세번째 항목으로 주어진 review_text에서,
    # 이제부터 불용어를 제거하는 루틴을 돌립니다.
    if remove_stopwords: # 만약에 이 함수를 쓸 때에 stopwords를 제거하라고 한다면
        words = review_text.split() # 현재 html제거하고, 알파벳으로만 구성된 리뷰글을 쪼개서 리스트로 만듦
        # 위의 split의 결과는 만약에 입력 리뷰글이 "I am a boy"였다면, [I, am, a, boy]으로 변환, words라는 리스트
        # words 리스트의 하나하나를 보면서, 만약에 그 하나가 set_stopwords에 속하지 않으면
        # 불용어가 아니다.
        words = [w for w in words if not w in list_stopwords]
        # 예를 들어 [i, am, a, boy]였고, 불용어가 am, a이었다면, 결과적으로 [I, boy]
        review_text = ' '.join(words)
        # 결과적으로 "I boy"가 된다.
    review_text = review_text.lower() # 대문자를 다 소문자로 바꾼다.
    return review_text
list_reviews = list(imdb_pd['review'])
print(list_reviews[0]) # 원래의 리뷰글
print(preprocessing(list_reviews[0])) # 쓰지않는 문장들 다 없


This movie is finally out on DVD in Italy (completely restored). I have seen this movie so many times and I find it even actual these days (2003) when Italy suffers again from a sort of brainwashing dictatorship (or the US for that matter). I am glad there are outcasts as the one played by Mastroianni in this movie who can sing out of tune; maybe they can teach the Sophia Lorens of this world how to be strong and fight to be recognised as human beings.<br /><br />Back to the movie: as most people here already mentioned the acting is wonderful but the audio background is astonishing. I must assume that unfortunately something is lost if you don't understand the Italian language but I can assure you that the show-off of machism, the distortion of reality in that ever-present radio-chronicle of the Hitler visit to Rome can really make you shiver!<br /><br />A masterpiece!<br /><br />
this movie finally dvd italy completely restored i seen movie many times i find even actual days italy suf